# Geographical Aggregation (Tourism)

> Geographical Hierarchical Forecasting on Australian Tourism Data

In many applications, a set of time series is hierarchically organized. Examples include the presence of geographic levels, products, or categories that define different types of aggregations. In such scenarios, forecasters are often required to provide predictions for all disaggregate and aggregate series. A natural desire is for those predictions to be **"coherent"**, that is, for the bottom series to add up precisely to the forecasts of the aggregated series.

In this notebook we present an example on how to use `HierarchicalForecast` to produce coherent forecasts between geographical levels. We will use the classic Australian Domestic Tourism (`Tourism`) dataset, which contains monthly time series of the number of visitors to each state of Australia.

We will first load the Tourism data and produce base forecasts using a diverse set of models, including `AutoETS` from `StatsForecast`, and machine learning models like `lightgbm` and `HistGradientBoostingRegressor` using `MLForecast`, as well as neural network models like `MLP` and `NBEATS` from `NeuralForecast`. We will then reconcile these base forecasts with several reconciliation algorithms from `HierarchicalForecast`. 

Finally, we show the performance is comparable with the results reported by the [Forecasting: Principles and Practice](https://otexts.com/fpp3/tourism.html) which uses the R package [fable](https://github.com/tidyverts/fable).

You can run these experiments using CPU or GPU with Google Colab.

<a href="https://colab.research.google.com/github/Nixtla/hierarchicalforecast/blob/main/nbs/examples/AustralianDomesticTourism.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
!pip install hierarchicalforecast statsforecast mlforecast datasetsforecast lightgbm sklearn neuralforecast

## 1. Load and Process Data

In this example we will use the [Tourism](https://otexts.com/fpp3/tourism.html) dataset from the [Forecasting: Principles and Practice](https://otexts.com/fpp3/) book.

The dataset only contains the time series at the lowest level, so we need to create the time series for all hierarchies.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
Y_df = pd.read_csv('https://raw.githubusercontent.com/Nixtla/transfer-learning-time-series/main/datasets/tourism.csv')
Y_df = Y_df.rename({'Trips': 'y', 'Quarter': 'ds'}, axis=1)
Y_df.insert(0, 'Country', 'Australia')
Y_df = Y_df[['Country', 'Region', 'State', 'ds', 'y']]
Y_df['ds'] = Y_df['ds'].str.replace(r'(\d+) (Q\d)', r'\1-\2', regex=True)
Y_df['ds'] = pd.PeriodIndex(Y_df["ds"], freq='Q').to_timestamp()
Y_df_first = Y_df.groupby(["Country", "Region", "State", "ds"], as_index=False).agg({"y":"sum"})
Y_df_first.head()

,Country,Region,State,ds,y
0,Australia,Adelaide,South Australia,1998-01-01,658.553895
1,Australia,Adelaide,South Australia,1998-04-01,449.853935
2,Australia,Adelaide,South Australia,1998-07-01,592.904597
3,Australia,Adelaide,South Australia,1998-10-01,524.242760
4,Australia,Adelaide,South Australia,1999-01-01,548.394105


The dataset can be grouped in the following non-strictly hierarchical structure.

In [ ]:
spec = [
    ['Country'],
    ['Country', 'State'], 
    #['Country', 'State', 'Region']
]

Using the `aggregate` function from `HierarchicalForecast` we can get the full set of time series.

In [ ]:
from hierarchicalforecast.utils import aggregate

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


In [ ]:
%%capture
Y_df, S_df, tags = aggregate(Y_df_first, spec)

In [ ]:
Y_df.head()

,unique_id,ds,y
0,Australia,1998-01-01,23182.197269
1,Australia,1998-04-01,20323.380067
2,Australia,1998-07-01,19826.640511
3,Australia,1998-10-01,20830.129891
4,Australia,1999-01-01,22087.353380


In [ ]:
S_df.iloc[:5, :5]

,unique_id,Australia/ACT,Australia/New South Wales,Australia/Northern Territory,Australia/Queensland
0,Australia,1.0,1.0,1.0,1.0
1,Australia/ACT,1.0,0.0,0.0,0.0
2,Australia/New South Wales,0.0,1.0,0.0,0.0
3,Australia/Northern Territory,0.0,0.0,1.0,0.0
4,Australia/Queensland,0.0,0.0,0.0,1.0


### Split Train/Test sets

We use the final two years (8 quarters) as test set.

In [ ]:
Y_test_df = Y_df.groupby('unique_id', as_index=False).tail(8)
Y_train_df = Y_df.drop(Y_test_df.index)

In [ ]:
Y_train_df.groupby('unique_id').size()

unique_id
Australia                       72
Australia/ACT                   72
Australia/New South Wales       72
Australia/Northern Territory    72
Australia/Queensland            72
Australia/South Australia       72
Australia/Tasmania              72
Australia/Victoria              72
Australia/Western Australia     72
dtype: int64

## 2. Computing base forecasts

The following cell computes the **base forecasts** for each time series in `Y_df` using the `ETS` model. Observe that `Y_hat_df` contains the forecasts but they are not coherent.

In [ ]:
Y_train_df = (
    Y_train_df
    .assign(
        hierarchy_number = lambda df: df.unique_id.str.count("/")
    )
    .assign(
        level = lambda df: np.where(
            df.hierarchy_number == 0, "Country",
            np.where(df.hierarchy_number == 1, "Country/State", "Country/State/Region")
        )
    )
    .drop(columns="hierarchy_number")
)

In [ ]:
from statsforecast.models import AutoETS
from statsforecast.core import StatsForecast
from typing import Tuple
from mlforecast import MLForecast
from neuralforecast import NeuralForecast

/Users/xwing/miniforge3/envs/hierarchicalforecast/lib/python3.10/site-packages/fs/__init__.py:4: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  __import__("pkg_resources").declare_namespace(__name__)  # type: ignore
/Users/xwing/miniforge3/envs/hierarchicalforecast/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-07-08 20:37:07,215	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the notebook server for rich notebook output.
2025-07-08 20:37:07,319	INFO util.py:154 -- Missing packages: ['ipywidgets']. Run `pip install -U ipywidgets`, then restart the noteb

In [ ]:
def fit_predict_any_model(model: StatsForecast | MLForecast | NeuralForecast, df: pd.DataFrame, h:int):
    if isinstance(model, StatsForecast):
        yhat = model.forecast(df=df, h=h, fitted=True)
        yfitted = model.forecast_fitted_values()
    if isinstance(model, MLForecast):
        model.fit(df, fitted=True)
        yhat = model.predict(new_df=df, h=h)
        yfitted = model.forecast_fitted_values()
    if isinstance(model, NeuralForecast):
        model.fit(df=df, val_size=h)
        yhat = model.predict()
        yfitted = model.predict_insample(step_size=h)
    return yhat, yfitted

In [ ]:
def create_predictions(model, hierarchy: np.ndarray, df: pd.DataFrame, h: int=8) -> Tuple[pd.DataFrame, pd.DataFrame]:
    Y_hat = []
    Y_fitted = []
    for level in hierarchy:
        df_helper = df.query("level==@level").drop(columns="level")
        yhat_level, yfitted_level = fit_predict_any_model(model, df_helper, h)
        Y_fitted.append(yfitted_level)
        Y_hat.append(yhat_level)
    Y_hat_df = pd.concat(Y_hat, ignore_index=True)
    Y_fitted_df = pd.concat(Y_fitted, ignore_index=True)
    return Y_hat_df, Y_fitted_df

In [ ]:
stat_model = StatsForecast(models=[AutoETS(season_length=4, model='ZZA')], freq='QS', n_jobs=-1)

In [ ]:
levels = Y_train_df.level.drop_duplicates()

In [ ]:
Y_hat_df_stats, Y_fitted_df_stats = create_predictions(stat_model, levels, Y_train_df)

In [ ]:
from sklearn.ensemble import HistGradientBoostingRegressor
from mlforecast.lag_transforms import ExpandingMean, RollingMean
from mlforecast.target_transforms import Differences

In [ ]:
ml_model = MLForecast(
    models = {
        'gbm':HistGradientBoostingRegressor()
    }, 
    freq='QS',
    target_transforms=[Differences([1, 4])],
    lags=[1, 2, 3, 4, 5, 6, 7, 8, 12],
    lag_transforms={  
        1: [ExpandingMean(), RollingMean(window_size=4)],
        4: [ExpandingMean(), RollingMean(window_size=2), RollingMean(window_size=4)],
        8: [RollingMean(window_size=4)]
    },
    date_features=['quarter', 'year']
)

In [ ]:
Y_hat_df_ml, Y_fitted_df_ml = create_predictions(ml_model, levels, Y_train_df)

In [ ]:
from neuralforecast.models import NBEATS
from neuralforecast.losses.pytorch import MAE

In [ ]:
neural_model = NeuralForecast(
    models=[
        NBEATS(
            h=8,
            input_size=16,
            mlp_units=[[256, 256], [256, 256], [256, 256]],
            learning_rate=1e-3,
            loss=MAE(),
            random_seed=42
        ),
    ],
    freq='QS'
)

Seed set to 42


In [ ]:
Y_hat_df_neural, Y_fitted_df_neural = create_predictions(neural_model, levels, Y_train_df)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name         | Type          | Params | Mode 
-------------------------------------------------------
0 | loss         | MAE           | 0      | train
1 | padder_train | ConstantPad1d | 0      | train
2 | scaler       | TemporalNorm  | 0      | train
3 | blocks       | ModuleList    | 620 K  | train
-------------------------------------------------------
620 K     Trainable params
408       Non-trainable params
620 K     Total params
2.482     Total estimated model params size (MB)
31        Modules in train mode
0         Modules in eval mode


Epoch 999: 100%|██████████| 1/1 [00:00<00:00, 31.09it/s, v_num=97, train_loss_step=136.0, train_loss_epoch=136.0, valid_loss=3.29e+3]

`Trainer.fit` stopped: `max_steps=1000` reached.


Epoch 999: 100%|██████████| 1/1 [00:00<00:00, 29.62it/s, v_num=97, train_loss_step=136.0, train_loss_epoch=136.0, valid_loss=3.29e+3]


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


...

Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 179.66it/s]


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 199.52it/s]

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs



Predicting DataLoader 0: 100%|██████████| 1/1 [00:00<00:00, 228.10it/s]


In [ ]:
Y_hat_df_stats.head(10)

,unique_id,ds,AutoETS
0,Australia,2016-01-01,25990.068004
1,Australia,2016-04-01,24458.490282
2,Australia,2016-07-01,23974.055984
3,Australia,2016-10-01,24563.454495
4,Australia,2017-01-01,25990.068004
5,Australia,2017-04-01,24458.490282
6,Australia,2017-07-01,23974.055984
7,Australia,2017-10-01,24563.454495
8,Australia/ACT,2016-01-01,553.037571
9,Australia/ACT,2016-04-01,568.944875


In [ ]:
Y_hat_df_ml.head(10)

,unique_id,ds,gbm
0,Australia,2016-01-01,26231.554679
1,Australia,2016-04-01,25140.463335
2,Australia,2016-07-01,24601.739685
3,Australia,2016-10-01,27117.217756
4,Australia,2017-01-01,28309.781195
5,Australia,2017-04-01,26760.885603
6,Australia,2017-07-01,27385.040786
7,Australia,2017-10-01,28526.713461
8,Australia/ACT,2016-01-01,678.274801
9,Australia/ACT,2016-04-01,654.820995


In [ ]:
Y_hat_df_neural.head(10)

,unique_id,ds,NBEATS
0,Australia,2016-01-01,27494.548828
1,Australia,2016-04-01,25739.089844
2,Australia,2016-07-01,25432.138672
3,Australia,2016-10-01,25137.085938
4,Australia,2017-01-01,26805.691406
5,Australia,2017-04-01,25052.847656
6,Australia,2017-07-01,22971.212891
7,Australia,2017-10-01,21669.326172
8,Australia/ACT,2016-01-01,523.254211
9,Australia/ACT,2016-04-01,536.496460


In [ ]:
Y_fitted_df_stats.head(10)

,unique_id,ds,y,AutoETS
0,Australia,1998-01-01,23182.197269,22521.177714
1,Australia,1998-04-01,20323.380067,21340.083713
2,Australia,1998-07-01,19826.640511,20316.741658
3,Australia,1998-10-01,20830.129891,20646.219131
4,Australia,1999-01-01,22087.353380,22170.448469
5,Australia,1999-04-01,21458.373285,20594.709951
6,Australia,1999-07-01,19914.192508,20568.113621
7,Australia,1999-10-01,20027.925640,20810.853242
8,Australia,2000-01-01,22339.294779,21822.581014
9,Australia,2000-04-01,19941.063482,20564.897505


In [ ]:
Y_fitted_df_ml.head(10)

,unique_id,ds,y,gbm
0,Australia,2002-04-01,20877.030328,20435.107447
1,Australia,2002-07-01,21299.143530,20634.874001
2,Australia,2002-10-01,21864.629491,20908.145840
3,Australia,2003-01-01,22061.776757,21780.833673
4,Australia,2003-04-01,21250.280204,20793.559395
5,Australia,2003-07-01,20685.211859,20665.129241
6,Australia,2003-10-01,21045.210989,21621.696711
7,Australia,2004-01-01,22949.053982,22182.313181
8,Australia,2004-04-01,21494.586465,20820.598508
9,Australia,2004-07-01,20799.516673,21724.500146


In [ ]:
Y_fitted_df_neural.head(10)

,unique_id,ds,cutoff,NBEATS,y
0,Australia,1998-01-01,1997-10-01,0.104817,23182.197266
1,Australia,1998-04-01,1997-10-01,0.133943,20323.380859
2,Australia,1998-07-01,1997-10-01,-0.005471,19826.640625
3,Australia,1998-10-01,1997-10-01,-0.161808,20830.130859
4,Australia,1999-01-01,1997-10-01,0.102119,22087.353516
5,Australia,1999-04-01,1997-10-01,0.128801,21458.373047
6,Australia,1999-07-01,1997-10-01,-0.032290,19914.193359
7,Australia,1999-10-01,1997-10-01,0.208161,20027.925781
8,Australia,2000-01-01,1999-10-01,21506.808594,22339.294922
9,Australia,2000-04-01,1999-10-01,20302.601562,19941.064453


In [ ]:
# TODO: Finish the reconciliation